In [1]:
import pandas as pd
import vaex as vx
import numpy as np
from datemerge import datemerge
from hyperlink import hyperlink
from to_excel_file import to_excel_file

topic_SP_list = ['THP Trà Xanh Không Độ','THP Nước tăng Lực Number 1','THP Sữa Đậu Nành Soya','THP Trà Thanh Nhiệt Dr. Thanh','THP Trà Sữa Macchiato Không Độ','THP Number 1 Active Chanh Muối','THP Number 1 Chanh & Dâu']
topic_list = ['Tân Hiệp Phát','Ban Lãnh Đạo Mới','THP Trà Xanh Không Độ','THP Nước tăng Lực Number 1','THP Sữa Đậu Nành Soya','THP Trà Thanh Nhiệt Dr. Thanh','THP Trà Sữa Macchiato Không Độ','THP Number 1 Active Chanh Muối','THP Number 1 Chanh & Dâu']
channel_list = ['Facebook', 'Fanpage', 'Tiktok', 'News']
sent_list = ['Positive', 'Neutral', 'Negative']
labels_list = ['HOẠT ĐỘNG FANPAGE','BAN LÃNH ĐẠO','SẢN PHẨM','THƯƠNG HIỆU/CÔNG TY','CHƯƠNG TRÌNH TÀI TRỢ','QUẢNG CÁO','TUYỂN DỤNG','CHIẾN DỊCH SỰ KIỆN']

CORP = pd.read_excel(r"D:\\Data Kompa\\Tân Hiệp Phát\\Monthly\\THPaug.xlsx", sheet_name='CORP') #(7SP, THP, BLD)
SP = pd.read_excel(r"D:\\Data Kompa\\Tân Hiệp Phát\\Monthly\\THPaug.xlsx", sheet_name='SP') #(7SP - chỉnh negative)

In [2]:
 slide_n_dict = {}

In [3]:
#Slide 6

channel_topic = CORP.pivot_table(index='Topic', columns='Channel', values='Id',aggfunc = 'count').fillna(0)
channel_topic['Total'] = channel_topic.sum(axis=1)
channel_topic = channel_topic.sort_values(by='Total',ascending = False).reindex(topic_list).reset_index()

channel_topic['Khác'] = channel_topic.drop(columns = channel_list+ ['Topic', 'Total']).sum(axis=1)
channel_topic = channel_topic[['Topic'] + channel_list + ['Khác','Total']] 
print(channel_topic)

slide_n_dict['Slide 6'] = channel_topic

Channel                           Topic  Facebook  Fanpage  Tiktok   News  \
0                         Tân Hiệp Phát      72.0     30.0     8.0  112.0   
1                      Ban Lãnh Đạo Mới       NaN      NaN     NaN    NaN   
2                 THP Trà Xanh Không Độ    2879.0    381.0    91.0   26.0   
3            THP Nước tăng Lực Number 1     437.0    149.0   102.0   32.0   
4                 THP Sữa Đậu Nành Soya     877.0     14.0     3.0   15.0   
5         THP Trà Thanh Nhiệt Dr. Thanh     158.0      0.0    16.0   17.0   
6        THP Trà Sữa Macchiato Không Độ      11.0      0.0     0.0    0.0   
7        THP Number 1 Active Chanh Muối       1.0      0.0     1.0    3.0   
8              THP Number 1 Chanh & Dâu       3.0      1.0     1.0    0.0   

Channel  Khác   Total  
0         4.0   226.0  
1         0.0     NaN  
2        10.0  3387.0  
3        21.0   741.0  
4         5.0   914.0  
5         7.0   198.0  
6         2.0    13.0  
7         1.0     6.0  
8         0.0

In [4]:
#Slide 7

slide_n_dict['Slide 7'] = datemerge(CORP.groupby(['PublishedDate', 'Topic']).agg({'Id': 'count'}).pivot_table(index='PublishedDate', values='Id', aggfunc='sum', columns='Topic'), '2024-07-26', '2024-08-25')
print(slide_n_dict['Slide 7'])

               THP Number 1 Active Chanh Muối  THP Number 1 Chanh & Dâu  \
PublishedDate                                                             
2024-07-26                                0.0                       0.0   
2024-07-27                                0.0                       0.0   
2024-07-28                                0.0                       0.0   
2024-07-29                                3.0                       1.0   
2024-07-30                                0.0                       0.0   
2024-07-31                                0.0                       1.0   
2024-08-01                                0.0                       0.0   
2024-08-02                                2.0                       1.0   
2024-08-03                                0.0                       0.0   
2024-08-04                                0.0                       0.0   
2024-08-05                                0.0                       0.0   
2024-08-06               

In [5]:
#Silde 8

nsr_chart_df = CORP.pivot_table(index='Topic', columns='Sentiment', values='Id', aggfunc='count').fillna(0)
nsr_chart_df['NSR'] = round((nsr_chart_df['Positive'] - nsr_chart_df['Negative']) / (nsr_chart_df['Positive'] + nsr_chart_df['Negative']), 3)
nsr_chart_df = nsr_chart_df.drop(sent_list, axis=1)

buzz_vertical_df = CORP.groupby('Topic').agg({'Id': 'count'}).pivot_table(index='Topic', values='Id', aggfunc='sum')

unique_voice_vertical_df = CORP.groupby('Topic').agg({'AuthorId': 'nunique'}).pivot_table(index='Topic', values='AuthorId', aggfunc='sum')

slide_8 = pd.concat([buzz_vertical_df, nsr_chart_df, unique_voice_vertical_df], axis=1).reindex(topic_list)
print(slide_8)
slide_n_dict['Slide 8'] = slide_8


                                    Id    NSR  AuthorId
Topic                                                  
Tân Hiệp Phát                    226.0  0.961     123.0
Ban Lãnh Đạo Mới                   NaN    NaN       NaN
THP Trà Xanh Không Độ           3387.0  0.946     471.0
THP Nước tăng Lực Number 1       741.0  0.876     368.0
THP Sữa Đậu Nành Soya            914.0  1.000     405.0
THP Trà Thanh Nhiệt Dr. Thanh    198.0  0.966      85.0
THP Trà Sữa Macchiato Không Độ    13.0  1.000       9.0
THP Number 1 Active Chanh Muối     6.0  1.000       6.0
THP Number 1 Chanh & Dâu           5.0  1.000       5.0


In [6]:
#Silde 9

topic_sentiment = CORP.pivot_table(index='Topic',columns = 'Sentiment', values = 'Id',aggfunc = 'count')
topic_sentiment['Total'] = topic_sentiment.sum(axis=1)
topic_sentiment = topic_sentiment.sort_values(by = 'Total', ascending = False).fillna(0).reset_index()
topic_sentiment = topic_sentiment[['Topic'] + sent_list + ['Total']]
print(topic_sentiment)

slide_n_dict['Slide 9'] = topic_sentiment

Sentiment                           Topic  Positive  Neutral  Negative   Total
0                   THP Trà Xanh Không Độ     143.0   3240.0       4.0  3387.0
1                   THP Sữa Đậu Nành Soya      23.0    891.0       0.0   914.0
2              THP Nước tăng Lực Number 1     181.0    548.0      12.0   741.0
3                           Tân Hiệp Phát     101.0    123.0       2.0   226.0
4           THP Trà Thanh Nhiệt Dr. Thanh     116.0     80.0       2.0   198.0
5          THP Trà Sữa Macchiato Không Độ       2.0     11.0       0.0    13.0
6          THP Number 1 Active Chanh Muối       3.0      3.0       0.0     6.0
7                THP Number 1 Chanh & Dâu       3.0      2.0       0.0     5.0


In [7]:
#Slide 10
df_list_CORP = []
for i in topic_list:
    brand = CORP[CORP['Topic'] == i]
    labels_df = brand.pivot_table(index='Labels1',columns='Sentiment',values='Id',aggfunc='count').reindex(labels_list).reindex(columns = sent_list)
    labels_df_pct_sent = round(labels_df.div(labels_df.sum().sum(),axis=0).fillna(0),3)
    labels_df_pct_sent.index.name = i
    print(labels_df_pct_sent)
    df_list_CORP.append(labels_df_pct_sent)
slide_n_dict['Slide 10'] = df_list_CORP

Sentiment             Positive  Neutral  Negative
Tân Hiệp Phát                                    
HOẠT ĐỘNG FANPAGE        0.022    0.111     0.000
BAN LÃNH ĐẠO             0.000    0.000     0.000
SẢN PHẨM                 0.000    0.000     0.000
THƯƠNG HIỆU/CÔNG TY      0.133    0.434     0.009
CHƯƠNG TRÌNH TÀI TRỢ     0.292    0.000     0.000
QUẢNG CÁO                0.000    0.000     0.000
TUYỂN DỤNG               0.000    0.000     0.000
CHIẾN DỊCH SỰ KIỆN       0.000    0.000     0.000
Sentiment             Positive  Neutral  Negative
Ban Lãnh Đạo Mới                                 
HOẠT ĐỘNG FANPAGE          0.0      0.0       0.0
BAN LÃNH ĐẠO               0.0      0.0       0.0
SẢN PHẨM                   0.0      0.0       0.0
THƯƠNG HIỆU/CÔNG TY        0.0      0.0       0.0
CHƯƠNG TRÌNH TÀI TRỢ       0.0      0.0       0.0
QUẢNG CÁO                  0.0      0.0       0.0
TUYỂN DỤNG                 0.0      0.0       0.0
CHIẾN DỊCH SỰ KIỆN         0.0      0.0       0.0


In [8]:
#Slide 3
slide_3 = []

#===SOV===
for i in topic_SP_list:
    CORP['Topic'] = CORP['Topic'].replace(i,'7 Sản phẩm')
SOV = CORP.groupby('Topic').agg({'Id':'count'})
slide_3.append(SOV)
print(SOV)

#===Sentiment===
sent = CORP.groupby('Sentiment').agg({'Id':'count'}).sort_values(by='Sentiment',ascending = False)
slide_3.append(sent)
print(sent)

#===Sentiment break by Topic===
topic_sent = CORP.pivot_table(index='Topic', columns='Sentiment', values='Id',aggfunc = 'count')
slide_3.append(topic_sent)
print(topic_sent)

slide_n_dict['Slide 3'] = slide_3

                 Id
Topic              
7 Sản phẩm     5264
Tân Hiệp Phát   226
             Id
Sentiment      
Positive    572
Neutral    4898
Negative     20
Sentiment      Negative  Neutral  Positive
Topic                                     
7 Sản phẩm           18     4775       471
Tân Hiệp Phát         2      123       101


In [45]:
#Slide 12 -> Slide 32


for i in topic_list:    
    df_list_SP = []
    SP_df_working = SP[SP['Topic'] == i]
    if SP_df_working.empty:
        print(f"No data available for topic: {i}")
        slide_n_dict[i] = None  # Or handle it as you prefer (e.g., append an empty list)
        continue  # Skip to the next topic if no data
    
    #=======Channel========
    SP_channel = SP_df_working.pivot_table(index='Channel', values = 'Id', aggfunc = 'count')
    SP_channel = SP_channel.sort_values(by = 'Id', ascending = False)
    df_list_SP.append(SP_channel)
    
    #=======Sentiment========
    SP_sentiment = SP_df_working.pivot_table(columns='Sentiment', values = 'Id', aggfunc = 'count').reindex(columns= sent_list).fillna(0)
    if SP_df_working.empty:
        continue
    else:
        SP_sentiment = round(SP_sentiment.div(SP_sentiment.sum(axis=1),axis=0),3)
    
    df_list_SP.append(SP_sentiment)
    
    #=======Labels========
    SP_labels = SP_df_working.pivot_table(index='Labels1', columns = 'Sentiment',values = 'Id', aggfunc = 'count').reindex(columns = sent_list).fillna(0)
    if SP_df_working.empty:
        continue
    else:
        SP_labels = round(SP_labels.div(SP_labels.sum().sum(),axis = 0).fillna(0),3)
    
    df_list_SP.append(SP_labels)
    
    slide_n_dict[i] = df_list_SP

slide_n_dict.keys()


No data available for topic: Ban Lãnh Đạo Mới


dict_keys(['Slide 6', 'Slide 7', 'Slide 8', 'Slide 9', 'Slide 10', 'Slide 3', 'Tân Hiệp Phát', 'Ban Lãnh Đạo Mới', 'THP Trà Xanh Không Độ', 'THP Nước tăng Lực Number 1', 'THP Sữa Đậu Nành Soya', 'THP Trà Thanh Nhiệt Dr. Thanh', 'THP Trà Sữa Macchiato Không Độ', 'THP Number 1 Active Chanh Muối', 'THP Number 1 Chanh & Dâu'])

In [46]:
to_excel_file('THPaugOutPut', slide_n_dict)